## Intro
This notebook aggregates the environmental data by event whereas before we were looking at the data by date. 

### Calculate number of locations that flooded

In [1]:
%matplotlib inline
from focus_intersection import subset_floods, flood_df, subset_locations
from get_server_data import get_table_for_variable, get_db_table_as_df, data_dir, db_filename
import pandas as pd
import numpy as np
import sqlite3
con = sqlite3.connect(db_filename)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
flood_locations = get_db_table_as_df('flood_locations')
len(flood_locations['location'].unique())

748

In this case we are just focusing on the subset of points that is in the downtown area thus the "subset_floods."

In [3]:
subset_locations = flood_locations['location']
flood_events = get_db_table_as_df('flood_events')
flood_events['event_date'] = pd.to_datetime(flood_events['event_date'])
flood_events['event_name'] = flood_events['event_name'].str.strip()
flood_events['dates'] = pd.to_datetime(flood_events['dates'])
subset_floods = flood_events[flood_events['location'].isin(subset_locations)]

In [4]:
grouped = subset_floods.groupby(['event_date', 'event_name'])

Get the number of dates the event spanned, the number of unique locations that were flooded during the event and the total number of locations flooded on all event dates. 

In [5]:
event_total_flooded = grouped.size()
event_dates = grouped['dates'].unique()
num_event_dates = grouped['dates'].nunique()
num_locations = grouped['location'].nunique()

In [6]:
event_df = pd.concat([event_dates, event_total_flooded, num_event_dates, num_locations], axis=1)
event_df.columns = ['dates', 'num_flooded', 'num_dates', 'num_locations']

In [7]:
event_df.tail()

,,dates,num_flooded,num_dates,num_locations
event_date,event_name,,,,
2016-07-31,Thunderstorm,[2016-07-31T00:00:00.000000000],8,1,8
2016-09-02,HERMINE,"[2016-09-03T00:00:00.000000000, 2016-09-04T00:...",40,2,40
2016-09-19,HEAVY RAIN,[2016-09-19T00:00:00.000000000],11,1,11
2016-09-20,unnamed,"[2016-09-21T00:00:00.000000000, 2016-09-22T00:...",101,4,101
2016-10-05,Hurricane Matthew,"[2016-10-05T00:00:00.000000000, 2016-10-08T00:...",111,7,109


### Where num_flooded does not equal num_locations _investigation_
Let's checkout one of the events where the num_flooded is greater than the num_locations. I would expect this to mean that one location was flooded on multiple days of the same event. But for '2014-07-24' the event is only on one day so that isn't what I expected.

In [8]:
idx = pd.IndexSlice
event_df.sort_index(inplace=True)
event_df.loc[idx['2014-07-24', :], :]

,,dates,num_flooded,num_dates,num_locations
event_date,event_name,,,,
2014-07-24,unnamed,"[2014-07-24T00:00:00.000000000, 2014-07-26T00:...",18,2,17


In [9]:
fl_724 = subset_floods[subset_floods['dates'] == '2014-07-24']
fl_724[fl_724['location'].duplicated(keep=False)]

,location,eventType,event_name,event_date,dates,times
444,HAMPTON BOULEVARD & W 21ST STREET,Flooded street,unnamed,2014-07-24,2014-07-24,20:29:25.000
445,HAMPTON BOULEVARD & W 21ST STREET,Flooded underpass,unnamed,2014-07-24,2014-07-24,20:29:25.000


So _here's_ what is happening. The location name is the same in two rows but there are two different event types: "flooded street" and "flooded underpass."
Now that I think about it, that may explain all the differences between the num_location and num_flooded columns. Let's try another one, this time one that spans more than one day: Irene.

In [10]:
event_df.sort_index(inplace=True)
event_df.loc[idx[:, 'Irene'], :]

,,dates,num_flooded,num_dates,num_locations
event_date,event_name,,,,
2011-08-27,Irene,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4,108


In [11]:
irene = subset_floods[subset_floods['event_name'].str.contains('Irene')].sort_values('location')
irene[irene['location'].duplicated(keep=False)]

,location,eventType,event_name,event_date,dates,times
181,1000 BLOCK OF E VIRGINIA BEACH BOULEVARD,Flooded street,Irene,2011-08-27,2011-08-28,04:18:01.000
182,1000 BLOCK OF E VIRGINIA BEACH BOULEVARD,Flooded underpass,Irene,2011-08-27,2011-08-28,04:18:01.000
185,E 21ST STREET & MONTICELLO AVENUE,Flooded underpass,Irene,2011-08-27,2011-08-28,08:44:35.000
104,E 21ST STREET & MONTICELLO AVENUE,Flooded street,Irene,2011-08-27,2011-08-27,06:08:00.000


Looks like that's it. Which is not what I was hoping to show. I was thinking that that tell me something about the variety of locations that were flooded over the days but that's not the case.

Let's try this one more time with Hurricane Joaquin

In [12]:
jqn = flood_df[flood_df['event'].str.contains('Joaquin')]

In [13]:
jqn[jqn['location'].duplicated(keep=False)]

,Unnamed: 0,﻿recid,location,event,eventType,xcoord,ycoord,dt,_date,_time


So that is interesting. Even though for hurricanes Matthew and Joaquin, the seven and six days respectively, none
of the flooded locations were reported twice for one event. Very interesting. So to me, this means we really should be looking at these things by 'event' and not by '\_date'. It also means that the num_locations col doesn't add any information. So imma delete that.

In [14]:
del event_df['num_locations']

### Looking into date in "event" column versus dates in "\_date" column
Sometimes the date listed in the "event" column is quite different than the date(s) listed in the "\_date" column. A good example of this is the event "unnamed (2/25/2016)" where the dates in the "\_date" column are 2016-05-05, 2016-05-06, and 2016-05-31"

In [15]:
flood_df[flood_df['event'].str.contains('2/25/2016')]

,Unnamed: 0,﻿recid,location,event,eventType,xcoord,ycoord,dt,_date,_time
760,760,4815,19TH BAY STREET & PLEASANT AVENUE,unnamed (2/25/2016),Flooded street,1.215646e+07,3.505946e+06,2016-05-05 20:43:53.000,2016-05-05,2016-05-05 20:43:53.000
761,761,4816,20TH BAY STREET & PLEASANT AVENUE,unnamed (2/25/2016),Flooded street,1.215680e+07,3.505887e+06,2016-05-05 20:44:41.000,2016-05-05,2016-05-05 20:44:41.000
762,762,4817,BOUSH STREET & W OLNEY ROAD,unnamed (2/25/2016),Flooded street,1.212921e+07,3.478803e+06,2016-05-05 20:46:10.000,2016-05-05,2016-05-05 20:46:10.000
763,763,4818,900 BLOCK OF E CHARLOTTE STREET,unnamed (2/25/2016),Flooded street,1.213223e+07,3.476292e+06,2016-05-05 20:51:34.000,2016-05-05,2016-05-05 20:51:34.000
764,764,4819,LLEWELLYN AVENUE & W VIRGINIA BEACH BOULEVARD,unnamed (2/25/2016),Flooded street,1.212906e+07,3.479121e+06,2016-05-05 20:52:17.000,2016-05-05,2016-05-05 20:52:17.000
765,765,4820,ORLEANS STREET & LAFAYETTE AVENUE,unnamed (2/25/2016),Flooded street,1.213787e+07,3.513103e+06,2016-05-06 20:51:41.000,2016-05-06,2016-05-06 20:51:41.000
766,766,4821,GRANBY STREET & LLEWELLYN AVENUE,unnamed (2/25/2016),Flooded street,1.213187e+07,3.490070e+06,2016-05-06 20:52:52.000,2016-05-06,2016-05-06 20:52:52.000
767,767,4823,DUKE STREET & W OLNEY ROAD,unnamed (2/25/2016),Flooded street,1.212885e+07,3.478992e+06,2016-05-31 08:45:33.000,2016-05-31,2016-05-31 08:45:33.000


So to look at this more closely, I will calculate the difference in days between the "event" column date and the dates in the "\_date" column.

When I tried to calculate the time between the 'event_date' and the 'dates' to see how far off these were I found that two events had the same 'event_date'. So I think it's appropriate to drop the 'unnamed' one based on the fact that the dates in the "\_date" column are further from the "event_date".

In [16]:
event_df.sort_index(inplace=True)
event_df.loc[idx['2016-07-30', :], :]

,,dates,num_flooded,num_dates
event_date,event_name,,,
2016-07-30,Thunderstorm,"[2016-07-30T00:00:00.000000000, 2016-08-02T00:...",13,3


In [17]:
i = event_df.loc[['2016-07-30', 'unnamed'],:].index
event_df.drop(i, inplace=True)
i = event_df.loc[idx['2014-09-13', "NAPSG"],:].index
event_df.drop(i, inplace=True)

In [18]:
event_df.reset_index(inplace=True)
event_df.set_index('event_date', inplace=True)
event_df

,event_name,dates,num_flooded,num_dates
event_date,,,,
2010-09-30,Nicole,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101,3
2010-12-16,Snow,[2010-12-16T00:00:00.000000000],2,1
2011-08-27,Irene,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4
2012-10-28,Sandy,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105,3
2013-10-09,Heavy Rain,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36,3
2014-05-16,Heavy Rain,[2014-05-16T00:00:00.000000000],35,1
2014-06-09,unnamed,[2014-06-09T00:00:00.000000000],1,1
2014-06-19,Thunderstorms,[2014-06-20T00:00:00.000000000],5,1
2014-07-03,Arthur,[2014-07-04T00:00:00.000000000],8,1


In [19]:
days_away = []
max_days = []
for d in event_df.index:
    try:
        ar = event_df.loc[d, 'dates'] - np.datetime64(d)
        ar = ar.astype('timedelta64[D]')
        days = ar / np.timedelta64(1, 'D')
        days_away.append(days)
        max_days.append(days.max())
    except ValueError:
        print d
event_df['days_away_from_event'] = days_away
event_df['max_days_away'] = max_days
print event_df.shape
event_df.head()

(44, 6)


,event_name,dates,num_flooded,num_dates,days_away_from_event,max_days_away
event_date,,,,,,
2010-09-30,Nicole,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101,3,"[0.0, 1.0, 4.0]",4.0
2010-12-16,Snow,[2010-12-16T00:00:00.000000000],2,1,[0.0],0.0
2011-08-27,Irene,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4,"[-3.0, 0.0, 1.0, 2.0]",2.0
2012-10-28,Sandy,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105,3,"[-1.0, 0.0, 1.0]",1.0
2013-10-09,Heavy Rain,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36,3,"[-1.0, 0.0, 1.0]",1.0


I don't trust the events that have higher days away so I will disregard any event with a "max_days_away" greater than 10. Five events fall under this category.

In [20]:
# event_df = event_df[event_df['max_days_away']<10]
print event_df.shape
event_df

(44, 6)


,event_name,dates,num_flooded,num_dates,days_away_from_event,max_days_away
event_date,,,,,,
2010-09-30,Nicole,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101,3,"[0.0, 1.0, 4.0]",4.0
2010-12-16,Snow,[2010-12-16T00:00:00.000000000],2,1,[0.0],0.0
2011-08-27,Irene,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4,"[-3.0, 0.0, 1.0, 2.0]",2.0
2012-10-28,Sandy,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105,3,"[-1.0, 0.0, 1.0]",1.0
2013-10-09,Heavy Rain,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36,3,"[-1.0, 0.0, 1.0]",1.0
2014-05-16,Heavy Rain,[2014-05-16T00:00:00.000000000],35,1,[0.0],0.0
2014-06-09,unnamed,[2014-06-09T00:00:00.000000000],1,1,[0.0],0.0
2014-06-19,Thunderstorms,[2014-06-20T00:00:00.000000000],5,1,[1.0],1.0
2014-07-03,Arthur,[2014-07-04T00:00:00.000000000],8,1,[1.0],1.0


## Now we'll get the rainfall, groundwater, tide, and wind for the events
First we need to get all of the data for the variables, aggregate it in various ways up to a daily time step and combine it into a dataframe

In [21]:
feature_df = pd.DataFrame()

In [30]:
feature_df.to_sql(con=con, name="dntwn_nor_daily_observations", if_exists="replace")

### Combine env. data with event data

In [31]:
def add_event_data(evnt_data, evnt_df, col_name, func, idx):
    res = func(evnt_data[col_name])
    evnt_df.loc[idx, col_name] = res
    return evnt_df

Now for each event we get an aggregate of the different variables for the given dates

In [32]:
event_df = pd.concat([event_df, pd.DataFrame(columns=feature_df.columns)])

for ind in event_df.index:
    # get the dates of the event and include the date in the "event" column
    ds = event_df.loc[ind, 'dates']
    ind = np.datetime64(ind)
    ds = np.append(ds, ind) if not ind in ds else ds
    
    event_data = feature_df.loc[ds]
 
    # combining data on event scale
    # get max over the event for these features
    max_cols = ['rain_hourly_max', 'rain_15_min_max', 'wind_vel_daily_avg', 'wind_vel_hourly_max_avg']
    
    # get mean over the event for these features
    mean_cols = ['gw_daily_avg', 'tide_daily_avg', 'wind_dir_daily_avg']
    
    # get sum over the event for these features
    sum_cols = ['rain_daily_sum']
    
    # do something else for these features
    other_cols = ['rain_prev_3_days', 'rain_hourly_max_time', 'rain_15_min_max_time', 'tide_rhrmx', 'tide_r15mx']
    
    
    for feat in feature_df.columns:
        if feat in max_cols:
            event_df = add_event_data(event_data, event_df, feat, np.max, ind)
        elif feat in mean_cols:
            event_df = add_event_data(event_data, event_df, feat, np.mean, ind)
        elif feat in sum_cols:
            event_df = add_event_data(event_data, event_df, feat, np.sum, ind)
        elif feat in other_cols:
            if feat=='rain_prev_3_days':
                event_df.loc[ind, feat] = event_data.loc[ind, feat]
            elif feat == 'rain_hourly_max_time' or feat == 'tide_rhrmx':
                max_ind = event_data['rain_hourly_max'].idxmax()
                event_df.loc[ind, feat] = event_data.loc[max_ind, feat]
            elif feat == 'rain_15_min_max_time' or feat == 'tide_r15mx':
                max_ind = event_data['rain_15_min_max'].idxmax()
                event_df.loc[ind, feat] = event_data.loc[max_ind, feat]
        else:
            raise ValueError("I don't know how to aggregate this variable on an event scale")
        
event_df.head()

,dates,days_away_from_event,event_name,gw_daily_avg,max_days_away,num_dates,num_flooded,rain_15_min_max,rain_15_min_max_time,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_prev_3_days,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg
2010-09-30,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...","[0.0, 1.0, 4.0]",Nicole,3.05116,4.0,3.0,101.0,0.67,2010-09-30 15:45:00,11.415,1.59,2010-09-30 04:15:00,3.46,1.4418,1.716,0.869,291.667,13.2,NaN
2010-12-16,[2010-12-16T00:00:00.000000000],[0.0],Snow,2.0974,0.0,1.0,2.0,0.03,2010-12-16 13:30:00,0.36,0.11,2010-12-16 14:15:00,0.06,-0.209,-0.459,-0.459,213,2.7,NaN
2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...","[-3.0, 0.0, 1.0, 2.0]",Irene,1.76129,2.0,4.0,110.0,0.33,2011-08-27 11:00:00,7.895,1.22,2011-08-27 11:30:00,0.425,1.10093,3.346,3.077,133,31,NaN
2012-10-28,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...","[-1.0, 0.0, 1.0]",Sandy,2.21188,1.0,3.0,105.0,0.21,2012-10-28 21:45:00,4.205,0.58,2012-10-28 22:15:00,0.495,2.75035,5.187,5.187,115.314,23.5047,22.1479
2013-10-09,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...","[-1.0, 0.0, 1.0]",Heavy Rain,1.66122,1.0,3.0,36.0,0.22,2013-10-10 01:15:00,4.32,0.5,2013-10-09 15:00:00,0.685,2.2397,4.4,3.711,39.3115,19.6308,18.4242


In [34]:
event_df.to_csv('{}event_data.csv'.format(data_dir))

In [35]:
event_df.iloc[[21,22],3:]

,gw_daily_avg,max_days_away,num_dates,num_flooded,rain_15_min_max,rain_15_min_max_time,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_prev_3_days,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg
2015-07-02,1.46882,9.0,2.0,7.0,1.42,2015-07-11 13:00:00,2.78667,4.14,2015-07-11 13:30:00,-2.33147e-15,0.516653,0.217,0.981,192.166,8.32003,10.9758
2015-07-11,1.28043,0.0,1.0,10.0,1.42,2015-07-11 13:00:00,2.48333,4.14,2015-07-11 13:30:00,-2.32684e-15,0.73125,0.217,0.981,114.762,5.31869,7.16792


### Combining with the non-flooding event data
First we have to combine all the dates in the "dates" column of the event_df into one array so we can filter those out of the overall dataset.

In [36]:
flooded_dates = [np.datetime64(i) for i in event_df.index]
flooded_dates = np.array(flooded_dates)
fl_event_dates = np.concatenate(event_df['dates'].tolist())
all_fl_dates = np.concatenate([fl_event_dates, flooded_dates])

In [37]:
non_flooded_records = feature_df[feature_df.index.isin(all_fl_dates) != True]
non_flooded_records['num_flooded'] = 0
non_flooded_records['flooded'] = False
non_flooded_records['event_name'] = np.nan
non_flooded_records['event_date'] = non_flooded_records.index
non_flooded_records.reset_index(drop=True, inplace=True)
non_flooded_records.head()

,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_15_min_max,rain_15_min_max_time,rain_prev_3_days,gw_daily_avg,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg,num_flooded,flooded,event_name,event_date
0,0.055,0.04,2010-01-01 22:45:00,0.02,2010-01-01 22:30:00,0.165,3.104508,0.650271,2.257,1.991,353.0,6.75,NaN,0,False,NaN,2010-01-01
1,0.000,0.00,NaT,0.00,NaT,0.145,3.414894,0.663292,NaN,NaN,321.0,16.90,NaN,0,False,NaN,2010-01-02
2,0.000,0.00,NaT,0.00,NaT,0.130,3.120831,-0.580583,NaN,NaN,295.0,13.90,NaN,0,False,NaN,2010-01-03
3,0.050,0.10,2010-01-04 13:30:00,0.10,2010-01-04 13:30:00,0.055,2.948871,-0.532312,0.709,0.709,306.0,8.50,NaN,0,False,NaN,2010-01-04
4,0.000,0.00,NaT,0.00,NaT,0.050,2.833045,0.024146,NaN,NaN,292.0,7.25,NaN,0,False,NaN,2010-01-05


Combine with flooded events

In [38]:
event_df.reset_index(inplace=True)
flooded_records = event_df
flooded_records['event_date'] = event_df['index']
flooded_records['flooded'] = True
flooded_records.head()

,index,dates,days_away_from_event,event_name,gw_daily_avg,max_days_away,num_dates,num_flooded,rain_15_min_max,rain_15_min_max_time,...,rain_hourly_max_time,rain_prev_3_days,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg,event_date,flooded
0,2010-09-30,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...","[0.0, 1.0, 4.0]",Nicole,3.05116,4.0,3.0,101.0,0.67,2010-09-30 15:45:00,...,2010-09-30 04:15:00,3.46,1.4418,1.716,0.869,291.667,13.2,NaN,2010-09-30,True
1,2010-12-16,[2010-12-16T00:00:00.000000000],[0.0],Snow,2.0974,0.0,1.0,2.0,0.03,2010-12-16 13:30:00,...,2010-12-16 14:15:00,0.06,-0.209,-0.459,-0.459,213,2.7,NaN,2010-12-16,True
2,2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...","[-3.0, 0.0, 1.0, 2.0]",Irene,1.76129,2.0,4.0,110.0,0.33,2011-08-27 11:00:00,...,2011-08-27 11:30:00,0.425,1.10093,3.346,3.077,133,31,NaN,2011-08-27,True
3,2012-10-28,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...","[-1.0, 0.0, 1.0]",Sandy,2.21188,1.0,3.0,105.0,0.21,2012-10-28 21:45:00,...,2012-10-28 22:15:00,0.495,2.75035,5.187,5.187,115.314,23.5047,22.1479,2012-10-28,True
4,2013-10-09,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...","[-1.0, 0.0, 1.0]",Heavy Rain,1.66122,1.0,3.0,36.0,0.22,2013-10-10 01:15:00,...,2013-10-09 15:00:00,0.685,2.2397,4.4,3.711,39.3115,19.6308,18.4242,2013-10-09,True


In [39]:
reformat = pd.concat([flooded_records, non_flooded_records], join='inner')
reformat.reset_index(inplace=True, drop=True)
reformat.head()

,event_name,gw_daily_avg,num_flooded,rain_15_min_max,rain_15_min_max_time,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_prev_3_days,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg,event_date,flooded
0,Nicole,3.05116,101.0,0.67,2010-09-30 15:45:00,11.415,1.59,2010-09-30 04:15:00,3.46,1.4418,1.716,0.869,291.667,13.2,NaN,2010-09-30,True
1,Snow,2.0974,2.0,0.03,2010-12-16 13:30:00,0.36,0.11,2010-12-16 14:15:00,0.06,-0.209,-0.459,-0.459,213,2.7,NaN,2010-12-16,True
2,Irene,1.76129,110.0,0.33,2011-08-27 11:00:00,7.895,1.22,2011-08-27 11:30:00,0.425,1.10093,3.346,3.077,133,31,NaN,2011-08-27,True
3,Sandy,2.21188,105.0,0.21,2012-10-28 21:45:00,4.205,0.58,2012-10-28 22:15:00,0.495,2.75035,5.187,5.187,115.314,23.5047,22.1479,2012-10-28,True
4,Heavy Rain,1.66122,36.0,0.22,2013-10-10 01:15:00,4.32,0.5,2013-10-09 15:00:00,0.685,2.2397,4.4,3.711,39.3115,19.6308,18.4242,2013-10-09,True


In [40]:
reformat['rain_hourly_max_time'] = reformat['rain_hourly_max_time'].astype('str')  # sqlite does not support native date format
reformat['rain_15_min_max_time'] = reformat['rain_15_min_max_time'].astype('str')
reformat.to_sql(name="for_model", con=con, index=False, if_exists='replace')